In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config Completer.use_jedi = False

In [3]:
%gui qt5

In [4]:
import napari

In [5]:
from pathlib import Path
import time

from pysmFISH.pipeline import Pipeline
from pysmFISH.utils import end_processing_file

In [24]:
from pysmFISH.io import load_general_zarr, load_raw_images
from pysmFISH.preprocessing import load_dark_image

In [18]:
experiment_fpath = Path('/Users/simone.codeluppi/Documents/data_analysis_jlabs_sc/test_stitching/data/LBEXP20210718_EEL_Mouse_448_2')
processing_engine = 'local'
dataset_path = experiment_fpath / '210830_09_49_20_LBEXP20210718_EEL_Mouse_448_2_img_data_dataset.parquet'
run_type = 're_run'
parsing_type = 'no_parsing'
parsed_zarr = experiment_fpath / 'LBEXP20210718_EEL_Mouse_448_2_img_data.zarr'

In [ ]:
# Because you are running the pipeline locally you should define the number of cores and memory/core
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        cores=2,
        memory='5GB')

In [10]:
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()  

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.worker - INFO - Codebook_Atto425 has None as codebook
distributed.worker - INFO - Codebook_Cy3 has None as codebook
distributed.worker - INFO - Codebook_Cy7 has None as codebook
distributed.worker - INFO - Codebook_DAPI has None as codebook
distributed.worker - INFO - Codebook_FITC has None as codebook
distributed.worker - INFO - Codebook_TxRed has None as codebook
distributed.worker - INFO - Codebook_Europium has None as codebook


In [12]:
running_pipeline.client.dashboard_link

'http://127.0.0.1:50493/status'

In [16]:
# Selected the fov to test processing with
fov = 262
round_num = 1
channel = running_pipeline.metadata['stitching_channel']

In [17]:
fov_subdataset = running_pipeline.data.select_specific_fov(running_pipeline.data.dataset,
                                          channel=channel,
                                          round_num=round_num,
                                          fov_num=fov)

In [19]:
# Load raw image
raw_img, metadata = load_raw_images(fov_subdataset.grp_name,
                                   parsed_zarr)

In [26]:
# Load dark image
dark_img = load_dark_image(running_pipeline.experiment_fpath)

In [20]:
# Visualise raw image
vw = napari.Viewer(title='raw image')
_ = vw.add_image(raw_img,name=str(fov),colormap='inferno')


In [21]:
# Processing parameters
running_pipeline.analysis_parameters

{'fish': {'PreprocessingFishFlatFieldKernel': [1, 100, 100],
  'PreprocessingFishFilteringSmallKernel': [1, 8, 8],
  'PreprocessingFishFilteringLaplacianKernel': [0.02, 0.01, 0.01],
  'CountingFishMinObjDistance': 1,
  'CountingFishMaxObjSize': 200,
  'CountingFishMinObjSize': 1,
  'CountingFishNumPeaksPerLabel': 20,
  'LargeObjRemovalPercentile': 95,
  'LargeObjRemovalMinObjSize': 100,
  'LargeObjRemovalSelem': 3},
 'both-beads': {'PreprocessingFishFilteringSmallKernel': [1, 8, 8],
  'PreprocessingFishFilteringLaplacianKernel': [0.02, 0.01, 0.01],
  'PreprocessingFishFlatFieldKernel': [1, 100, 100],
  'CountingFishMinObjDistance': 5,
  'CountingFishMaxObjSize': 600,
  'CountingFishMinObjSize': 10,
  'CountingFishNumPeaksPerLabel': 1,
  'LargeObjRemovalPercentile': 95,
  'LargeObjRemovalMinObjSize': 100,
  'LargeObjRemovalSelem': 3},
 'staining': {'PreprocessingStainingFlatFieldKernel': [2, 100, 100]},
 'fresh-tissue': {'nuclei': {'PreprocessingFreshNucleiLargeKernelSize': [5,
    50,


In [22]:
# Running functions
running_pipeline.running_functions

{'fish_channels_preprocessing': 'standard_not_norm_preprocessing',
 'fish_channels_dots_calling': 'osmFISH_peak_based_detection_fast',
 'fresh_sample_reference_preprocessing': 'large_beads_preprocessing',
 'fresh_sample_reference_dots_calling': 'osmFISH_peak_based_detection_fast',
 'fresh_sample_nuclei_preprocessing': 'fresh_nuclei_filtering',
 'reference_channels_preprocessing': 'both_beads_preprocessing',
 'reference_channels_dots_calling': 'both_beads_peak_based_detection'}

In [23]:
# Selected the function to use
from pysmFISH.preprocessing import both_beads_preprocessing

In [28]:
filtering_out = both_beads_preprocessing(fov_subdataset.grp_name,
                                        parsed_zarr,
                                        running_pipeline.analysis_parameters['both-beads'],
                                        dark_img)

In [32]:
# Visualise filtered image
vw = napari.Viewer(title='filtered image')
_ = vw.add_image(filtering_out[0][-1],name='filtered fov '+str(fov),colormap='inferno')



In [33]:
from pysmFISH.dots_calling import both_beads_peak_based_detection

In [34]:
counts_df = both_beads_peak_based_detection(filtering_out[0][-1], 
                                            fov_subdataset,
                                           running_pipeline.analysis_parameters['both-beads'])

In [35]:
coords = counts_df.loc[:,['r_px_original','c_px_original']].to_numpy()

In [36]:
# Visualise filtered image
vw = napari.Viewer(title='filtered image and counts')
_ = vw.add_image(filtering_out[0][-1],name='filtered fov '+str(fov),colormap='inferno')
_ = vw.add_points(coords,name='counts'+ str(fov), symbol='+', face_color='green', size=5, opacity=0.8)
